In [1]:
"""Save AWG values for the best data from the DRAG."""

import os
import pickle
from datetime import datetime

import numpy as np
import tensorflow as tf
from c3po.utils import log_setup
from c3po.tf_utils import tf_abs, tf_ave
from c3po.qt_utils import basis, xy_basis, perfect_gate, single_length_RB

from scipy.linalg import expm
import c3po.hamiltonians as hamiltonians
from c3po.simulator import Simulator as Sim
from c3po.optimizer import Optimizer as Opt
from c3po.experiment import Experiment as Exp
from single_qubit import create_chip_model, create_generator, create_gates

# Script parameters
lindbladian = False
IBM_angles = False
search_fid = 'unit'  # 'state' 'unit' 'orbit'
pulse_type = 'drag'  # 'gauss' 'drag' 'pwc'
sim_res = 3e11
awg_res = 2.4e9
awg_sample = 1 / awg_res
sample_numbers = np.arange(4, 20, 1)
logdir = log_setup("/tmp/c3logs/")

# System
qubit_freq = 5.1173e9 * 2 * np.pi
qubit_anhar = -315.513734e6 * 2 * np.pi
qubit_lvls = 4
drive_ham = hamiltonians.x_drive
v_hz_conversion = 1

1 Physical GPUs, 1 Logical GPUs


In [5]:
base_dir = '/home/usersFWM/froy/Documents/PHD/'
dir_name_drag = 'hockey_open/2019-11-07_13:15/'
savefile_drag = 'drag_N4.pickle'
with open('{}{}{}'.format(base_dir,
                          dir_name_drag, 
                          savefile_drag), 'rb') as file:
    unit_inf = pickle.load(file)
    times = pickle.load(file)
    best_params = pickle.load(file)

In [7]:
AWG_values = []
for indx in range(len(times)):
    print("#: ", indx)
    # Update experiment time
    sample_num = sample_numbers[indx]
    t_final = times[indx]
    params = best_params[indx]
    
    # Create generator and pulse instructions
    gen = create_generator(sim_res, awg_res, v_hz_conversion, logdir=logdir)
    gen.devices['awg'].options = 'drag'
    gates = create_gates(t_final, v_hz_conversion, qubit_freq, qubit_anhar)

    gateset_opt_map = [
        [('X90p', 'd1', 'gauss', 'amp')],
        [('X90p', 'd1', 'gauss', 'freq_offset')],
        [('X90p', 'd1', 'gauss', 'delta')],
        [('X90p', 'd1', 'gauss', 'xy_angle')]
    ]
    
    gates.set_parameters(params, gateset_opt_map)
    awg = gen.devices['awg']
    instr = gates.instructions["X90p"]
    awg_signal = awg.create_IQ(instr.comps['d1'], instr.t_start, instr.t_end)
    AWG_values.append(awg_signal)

#:  0
#:  1
#:  2
#:  3
#:  4
#:  5
#:  6
#:  7
#:  8
#:  9
#:  10
#:  11
#:  12
#:  13
#:  14
#:  15


In [9]:
AWG_values

[{'inphase': <tf.Tensor: id=119, shape=(4,), dtype=float64, numpy=array([3.23172534e+08, 1.52086884e+09, 1.41063920e+09, 2.90373817e+08])>,
  'quadrature': <tf.Tensor: id=120, shape=(4,), dtype=float64, numpy=array([ 4.27402538e+08,  1.42016504e+09, -1.52970841e+09, -4.50329283e+08])>},
 {'inphase': <tf.Tensor: id=242, shape=(5,), dtype=float64, numpy=
  array([7.83698050e+07, 8.18875108e+08, 1.70118328e+09, 8.24269297e+08,
         7.96145650e+07])>,
  'quadrature': <tf.Tensor: id=243, shape=(5,), dtype=float64, numpy=
  array([ 2.30238297e+08,  9.99038440e+08,  4.60291049e+06, -9.94592566e+08,
         -2.29810837e+08])>},
 {'inphase': <tf.Tensor: id=365, shape=(6,), dtype=float64, numpy=
  array([1.36900375e+08, 6.30131855e+08, 1.08359900e+09, 1.08282472e+09,
         6.29333748e+08, 1.36738392e+08])>,
  'quadrature': <tf.Tensor: id=366, shape=(6,), dtype=float64, numpy=
  array([ 1.25567770e+08,  6.18715148e+08,  5.99936938e+08, -6.01333303e+08,
         -6.19526935e+08, -1.2574414

In [12]:
with open('best_drag_awg.pickle', 'wb') as file:
    pickle.dump(AWG_values, file)